In [146]:
import torch

device = 0 if torch.cuda.is_available() else -1

import pandas as pd

df =pd.read_csv("../../dataset/romanian_political_articles_v1.csv")

df = df.dropna(subset=["maintext", "source_domain"])

In [147]:
import re
import spacy

nlp = spacy.load("ro_core_news_sm")

def remove_quotes(text):
    return re.sub(r'[\'"„”][^\'"„”]{1,300}?[\'"”]', '', text)

def remove_html(text):
    return re.sub(r"<.*?>", "", text)

def remove_digits_keep_years(text):
    return re.sub(r'\b(?!20\d{2})\d+\b', '', text)

def remove_urls(text):
    return re.sub(r'http\S+', '', text)

def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

def remove_weird_punctuation(text):
    return re.sub(r'[\*\•\·@~^_`+=\\|]', '', text)

def preprocess_for_romanian_models(text):
    return text.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

def preprocess_text(text : str):
    if not text or len(text.strip()) == 0:
        return ""

    text = preprocess_for_romanian_models(text)

    text = remove_html(text)

    text = remove_quotes(text)

    text = remove_urls(text)

    text = remove_digits_keep_years(text)

    text = remove_weird_punctuation(text)

    text = normalize_whitespace(text)

    return text.strip()

df['cleantext'] = df['maintext'].progress_apply(preprocess_text)

df = df[df['cleantext'].str.split().str.len() > 100]

df.head()

100%|██████████| 4691/4691 [00:01<00:00, 2531.03it/s]


,url,title,date_publish,description,maintext,source_domain,authors,cleantext
0,https://www.realitatea.net/stiri/politica/ion-...,Ion Cristoiu: Ilie Bolojan și rețeaua sa duc o...,2025-04-08 08:38:37,Ilie Bolojan si reteaua sa duc o politica anti...,"""Domnul Sprînceană a intervenit, nu știu ce l-...",www.realitatea.net,Realitatea.NET,"""Domnul Sprînceană a intervenit, nu știu ce l-..."
1,https://www.realitatea.net/stiri/politica/scan...,Scandalurile prin care a devenit celebru candi...,2025-04-13 13:09:59,Tupeu incredibil din partea lui Nicusor Dan! S...,Activistă și ea din zona soroșistă ce a venit ...,www.realitatea.net,Georgiana Balaban,Activistă și ea din zona soroșistă ce a venit ...
2,https://www.realitatea.net/stiri/politica/crin...,"Crin Antonescu: „Nicușor dă prea puțină apă, P...",2025-04-12 20:54:54,",,Nicusor Dan da prea putina apa si Ponta prea...","Crin Antonescu: „Ei promit lapte și miere, dar...",www.realitatea.net,Georgiana Balaban,Crin Antonescu: „Domnul Nicușor Dan după ce a ...
5,https://www.realitatea.net/stiri/politica/geor...,George Simion: „Ideea de tur doi înapoi înseam...,2025-04-12 21:50:26,",,Ideea de tur doi inapoi inseamna revenirea l...",Totul pentru a evita o situație asemănătoare c...,www.realitatea.net,Georgiana Balaban,Totul pentru a evita o situație asemănătoare c...
6,https://www.realitatea.net/stiri/politica/flor...,Florin Zamfirescu: „Oamenii nu mai au cu cine ...,2025-04-13 08:54:16,Florin Zamfirescu a vorbit in exclusivitate la...,Actorul spune că românii sunt îngenuncheați și...,www.realitatea.net,Georgiana Balaban,Actorul spune că românii sunt îngenuncheați și...


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, pipeline
import spacy
from tqdm import tqdm
tqdm.pandas()

nlp_ro = spacy.load("ro_core_news_sm")

def get_romanian_ner_nlp_pipeline():
    romanian_ner_model = "dumitrescustefan/bert-base-romanian-ner"
    tokenizer = AutoTokenizer.from_pretrained(romanian_ner_model, model_max_length=512)

    config = AutoConfig.from_pretrained(romanian_ner_model)
    config.id2label = {
        0: 'O', 1: 'B-PERSON', 2: 'I-PERSON', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-GPE', 6: 'I-GPE', 7: 'B-LOC',
        8: 'I-LOC', 9: 'B-NAT_REL_POL', 10: 'I-NAT_REL_POL', 11: 'B-EVENT', 12: 'I-EVENT', 13: 'B-LANGUAGE',
        14: 'I-LANGUAGE', 15: 'B-WORK_OF_ART', 16: 'I-WORK_OF_ART', 17: 'B-DATETIME', 18: 'I-DATETIME',
        19: 'B-PERIOD', 20: 'I-PERIOD', 21: 'B-MONEY', 22: 'I-MONEY', 23: 'B-QUANTITY', 24: 'I-QUANTITY',
        25: 'B-NUMERIC', 26: 'I-NUMERIC', 27: 'B-ORDINAL', 28: 'I-ORDINAL', 29: 'B-FACILITY', 30: 'I-FACILITY'
    }

    model = AutoModelForTokenClassification.from_pretrained(romanian_ner_model, config=config)

    return pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy='simple', device=device)

ner_pipeline = get_romanian_ner_nlp_pipeline()

def is_pronoun(word):
    return any(token.pos_ == "PRON" for token in nlp_ro(word))

def extract_named_entities(text):
    entities = set()

    try:
        doc = nlp_ro(text)

        for sent in doc.sents:
            sent_entities = ner_pipeline(sent.text.strip())

            for entity in sent_entities:
                entity_group = entity['entity_group']
                word = entity['word'].strip(" .,:;\"'?!/><)(*&^%$@+-=_-”„“").replace("##", "")

                if entity_group in ['PERSON', 'GPE'] and not is_pronoun(word):
                    entities.add(word)

    except Exception as e:
        print("NER failed:", e)

    return list(entities)

df['ner'] = df['cleantext'].progress_apply(extract_named_entities)
df.head()

Device set to use cpu
  6%|▌         | 228/4052 [04:23<1:31:44,  1.44s/it]

In [ ]:
from transformers import AutoModelForSequenceClassification

def get_romanian_sentiment_pipeline():
    romanian_sentiment_model = "DGurgurov/xlm-r_romanian_sentiment"
    tokenizer = AutoTokenizer.from_pretrained(romanian_sentiment_model, model_max_length=512)

    config = AutoConfig.from_pretrained(romanian_sentiment_model)
    config.id2label = {0: "negative", 1: "positive"}

    model = AutoModelForSequenceClassification.from_pretrained(romanian_sentiment_model, config=config)
    print(model.config.id2label)

    return pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

sentiment_pipeline = get_romanian_sentiment_pipeline()

def get_sentiment(sentiment_output, neutral_threshhold=0.63):
    label = sentiment_output['label']
    score = sentiment_output['score']

    if score <= neutral_threshhold:
        return 'neutral'

    return label

def get_ner_sentiments(text, ner_entities, max_tokens=512):
    sentiments = []

    doc = nlp_ro(text)

    for entity in ner_entities:
        name_lower = entity.lower()

        related_sentences = [sent.text.strip() for sent in doc.sents if name_lower in sent.text.lower()]

        if not related_sentences:
            continue

        try:
            results = sentiment_pipeline(related_sentences, truncation=True, max_length=max_tokens)

            for sentence, sentiment in zip(related_sentences, results):
                sentiment_label = get_sentiment(sentiment)

                sentiments.append({
                    'entity': entity,
                    'sentence': sentence,
                    'sentiment': sentiment_label,
                    'score': round(sentiment['score'], 3)
                })
        except Exception as e:
            print("Sentiment analysis failed:", e)
            continue

    return sentiments

df['ner_sentiments'] = df.progress_apply(lambda row: get_ner_sentiments(row['cleantext'], row['ner']), axis=1)
df.head()

Device set to use cpu


{0: 'negative', 1: 'positive'}


100%|██████████| 4016/4016 [7:43:12<00:00,  6.92s/it]   


,url,title,date_publish,description,maintext,source_domain,authors,cleantext,ner,ner_sentiments
1,https://www.realitatea.net/stiri/politica/scan...,Scandalurile prin care a devenit celebru candi...,2025-04-13 13:09:59,Tupeu incredibil din partea lui Nicusor Dan! S...,Activistă și ea din zona soroșistă ce a venit ...,www.realitatea.net,Georgiana Balaban,Activistă și ea din zona soroșistă ce a venit ...,"[PERIOD, Bucureștiul, domnul Nicușor Dan, pers...","[{'entity': 'PERIOD', 'sentence': 'Activistă ș..."
2,https://www.realitatea.net/stiri/politica/crin...,"Crin Antonescu: „Nicușor dă prea puțină apă, P...",2025-04-12 20:54:54,",,Nicusor Dan da prea putina apa si Ponta prea...","Crin Antonescu: „Ei promit lapte și miere, dar...",www.realitatea.net,Georgiana Balaban,Crin Antonescu Ei promit lapte și miere dar nu...,"[băieți, candidați, Mahomed, personaj, oamenii...","[{'entity': 'băieți', 'sentence': 'dar poate c..."
3,https://www.realitatea.net/stiri/politica/lasc...,Lasconi: „Nicușor Dan mi-a cerut să mă retrag ...,2025-04-12 19:02:32,",,Nicusor Dan mi-a cerut sa ma duc la Stejarii...",Candidata la prezidențiale trădată de propriul...,www.realitatea.net,Georgiana Balaban,Candidata la prezidențiale trădată de propriul...,"[Capitalei, Ciucă, Zaherman, edilul, Nicușor D...","[{'entity': 'Capitalei', 'sentence': 'Candidat..."
5,https://www.realitatea.net/stiri/politica/geor...,George Simion: „Ideea de tur doi înapoi înseam...,2025-04-12 21:50:26,",,Ideea de tur doi inapoi inseamna revenirea l...",Totul pentru a evita o situație asemănătoare c...,www.realitatea.net,Georgiana Balaban,Totul pentru a evita o situație asemănătoare c...,"[George Simion, domnul Călin Georgescu, președ...","[{'entity': 'George Simion', 'sentence': 'PERI..."
6,https://www.realitatea.net/stiri/politica/flor...,Florin Zamfirescu: „Oamenii nu mai au cu cine ...,2025-04-13 08:54:16,Florin Zamfirescu a vorbit in exclusivitate la...,Actorul spune că românii sunt îngenuncheați și...,www.realitatea.net,Georgiana Balaban,Actorul spune că românii sunt îngenuncheați și...,"[popor, România, Ponta, domne omul, Florin Zam...","[{'entity': 'popor', 'sentence': 'Actorul spun..."
